In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time

In [99]:
img = cv2.imread('test2.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [100]:
cv2.imshow('img', gray)
cv2.waitKey(0)

-1

In [82]:
gray

array([[47, 50, 66, ..., 49, 52, 47],
       [47, 52, 70, ..., 49, 53, 48],
       [51, 54, 71, ..., 49, 51, 47],
       ...,
       [16, 16, 14, ..., 22, 16, 14],
       [17, 16, 14, ..., 18, 15, 15],
       [17, 17, 15, ..., 14, 14, 17]], dtype=uint8)

In [83]:
Fy = convolve(gray)

In [84]:
cv2.imshow('img', Fy)
cv2.waitKey(0)

-1

In [96]:
# lines = cv2.HoughLines(Fy2, 1, np.pi/180, 624)
# lines = detectLines(Fy, minVotes = 80, rRes = 10, tRes = np.pi / 20, best = 3)
lines = detectLines(Fy, minVotes = 80, rRes = 1, tRes = np.pi / 180, best = 1)
temp = img.copy()
print(lines)

for l in lines:
    rho, theta, votes = l
    b = np.cos(theta)
    a = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))

    cv2.line(temp,(x1,y1),(x2,y2),(0, 0, 255),2)

# cv2.imwrite('test3.jpg',Fy)
cv2.imshow('img', temp)
cv2.waitKey(0)

[[-329.            3.07177948  211.        ]]


-1

In [86]:
start = time.perf_counter()
for i in range(10):
    lines = detectLines(Fy, minVotes = 100)
print(f'time per image: {(time.perf_counter() - start) / 10} seconds')

time per image: 1.277159440000105 seconds


## FUNCTIONS TO USE

combined into general pipeline to go from image file name to draw lines on it

In [87]:
def convolve(img, height = 9, sigma = 1):
    Gy = np.array([1 / (sigma * np.sqrt(2 * np.pi)) * np.e**(-y**2 / (2 * sigma**2)) for y in range(-height//2+1, height//2+1)])
    Gy = np.expand_dims(Gy, axis = 1) / np.sum(Gy)
    Gy = Gy - np.ones(height) / height
    return cv2.filter2D(img, -1, Gy)

In [43]:
def detectLines(img, minVotes = 10, rRes = 1, tRes = np.pi / 180, minBrightness = 240, best = None):
    height, width = img.shape
    ans = dict()
    for r in range(height):
        for c in range(width):
            if img[r,c] < minBrightness:
                continue
            for i in range(int(np.pi / tRes)):
                rho = (r * np.cos(i * tRes) + c * np.sin(i * tRes)) // rRes * rRes
                if rho in ans.keys():
                    ans[rho][i] = ans[rho].get(i, 0) + 1
                else:
                    ans[rho] = dict()
                    ans[rho][i] = 1
                    
    # create output array
    out = []
    for rho, d in ans.items():
        for theta, votes in d.items():
            if votes >= minVotes:
                out.append([rho, theta * tRes, ans[rho][theta]])
    if len(out) == 0:
        return detectLines(img, minVotes//3, rRes, tRes, minBrightness, best = best)
    out = np.array(out)
    
    if best == None:
        return out[(-out)[:, 2].argsort()]
    return out[(-out)[:, 2].argsort()][:best]
    
# print(detectLines(Fy, 80))

In [73]:
def everything(file = 'test.jpg'):
    img = cv2.imread(file)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    Fy = convolve(gray)
    lines = detectLines(Fy, minVotes = 80, rRes = 1, tRes = np.pi / 180, best = 1)
    temp = img.copy()

    for l in lines:
        rho, theta, votes = l
        b = np.cos(theta)
        a = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))

        cv2.line(temp,(x1,y1), (x2,y2), (0,0,255), 2)

    cv2.imshow('img', temp)
    cv2.waitKey(0)

everything('./sloth/cellos_crop/image19.jpg')


In [75]:
img = cv2.imread('./sloth/cellos_crop/image8.jpg')
cv2.imshow('img', img)
cv2.waitKey(0)

-1